In [1]:
import joblib
import numpy as np
import torch

import modularised_utils as mut
import matplotlib.pyplot as plt

import opt_utils as oput

import Linear_Additive_Noise_Models as lanm
import operations as ops
from scipy.linalg import sqrtm

import params

np.random.seed(0)

In [2]:
experiment = 'synth1'

In [3]:
# Define the radius of the Wasserstein balls (epsilon, delta) and the size for both models.
epsilon         = params.radius[experiment][0]
ll_num_envs     = params.n_envs[experiment][0]

delta           = params.radius[experiment][1]
hl_num_envs     = params.n_envs[experiment][1]

# Define the number of samples per environment. Currently every environment has the same number of samples
num_llsamples   = params.n_samples[experiment][0]
num_hlsamples   = params.n_samples[experiment][1]

In [4]:
Dll = mut.load_samples(experiment)[None][0] 
Gll = mut.load_ll_model(experiment)[0]
Ill = mut.load_ll_model(experiment)[1]


Dhl = mut.load_samples(experiment)[None][1] 
Ghl = mut.load_hl_model(experiment)[0]
Ihl = mut.load_hl_model(experiment)[1]

omega = mut.load_omega_map(experiment)

In [5]:
ll_coeffs = mut.get_coefficients(Dll, Gll)
hl_coeffs = mut.get_coefficients(Dhl, Ghl) 

In [6]:
# # [Not suggested] In case we want to explore also the interventional --> worse estimation!
# Dlls, Dhls = [], []
# for dpair in list(mut.load_samples(experiment).values()):
#     Dlls.append(dpair[0])
#     Dhls.append(dpair[1])
    
# ll_coeffs = mut.get_coefficients(Dlls, Gll)
# hl_coeffs = mut.get_coefficients(Dhls, Ghl) 

In [7]:
U_ll_hat, mu_U_ll_hat, Sigma_U_ll_hat = mut.lan_abduction(Dll, Gll, ll_coeffs)
U_hl_hat, mu_U_hl_hat, Sigma_U_hl_hat = mut.lan_abduction(Dhl, Ghl, hl_coeffs)

In [8]:
LLmodels = {}
for iota in Ill:
    LLmodels[iota] = lanm.LinearAddSCM(Gll, ll_coeffs, iota)
    
HLmodels, Dhl_samples = {}, {}
for eta in Ihl:
    HLmodels[eta] = lanm.LinearAddSCM(Ghl, hl_coeffs, eta)

In [10]:
LLmodels[None].compute_mechanism()

array([[1.        , 0.28018097, 0.06072652],
       [0.        , 1.        , 0.21674035],
       [0.        , 0.        , 1.        ]])

### Barycenter

In [9]:
L_matrices = []  # List of L_i matrices
for iota in Ill:
    L_matrices.append(LLmodels[iota].compute_mechanism())

H_matrices = []  # List of H_i matrices
for eta in Ihl:
    H_matrices.append(HLmodels[eta].compute_mechanism())

mu_bary_L, Sigma_bary_L = oput.compute_gauss_barycenter(L_matrices, mu_U_ll_hat, Sigma_U_ll_hat)
mu_bary_H, Sigma_bary_H = oput.compute_gauss_barycenter(H_matrices, mu_U_hl_hat, Sigma_U_hl_hat)

print("Low-level barycenter Mean:", mu_bary_L)
print("Low-level barycenter Covariance:", Sigma_bary_L)
print( )
print("High-level barycenter Mean:", mu_bary_H)
print("High-level barycenter Covariance:", Sigma_bary_H)

Low-level barycenter Mean: [-0.00678588 -0.01069607 -0.00015191]
Low-level barycenter Covariance: [[1.04033442 0.28379336 0.03931234]
 [0.28379336 2.0108559  0.21143766]
 [0.03931234 0.21143766 0.99009455]]

High-level barycenter Mean: [ 0.0042843  -0.00863504]
High-level barycenter Covariance: [[1.35385779 0.58226716]
 [0.58226716 0.97012678]]


In [17]:
V                 = oput.sample_projection(mu_U_ll_hat.shape[0], mu_U_hl_hat.shape[0], use_stiefel=False)
mu_bary_L_proj    = V @ mu_bary_L
Sigma_bary_L_proj = V @ Sigma_bary_L @ V.T

monge, A = oput.monge_map(mu_bary_L_proj, Sigma_bary_L_proj, mu_bary_H, Sigma_bary_H)
T        = V.T @ A

In [95]:
# Ambiguity set construction: Based on epsilon and delta include distribution (as many as the num_envs) that
# pass the "gelbrich" test.
ll_moments = mut.sample_moments_U(mu_hat    = mu_U_ll_hat,
                                  Sigma_hat = Sigma_U_ll_hat,
                                  bound     = epsilon,
                                  num_envs  = ll_num_envs)

A_ll       = mut.sample_distros_Gelbrich(ll_moments) #Low-level: A_epsilon


hl_moments = mut.sample_moments_U(mu_hat    = mu_U_hl_hat,
                                  Sigma_hat = Sigma_U_hl_hat,
                                  bound     = delta,
                                  num_envs  = hl_num_envs)

A_hl       = mut.sample_distros_Gelbrich(hl_moments) #High-level A_delta

In [10]:
abstraction_errors             = {}
abstraction_env_errors         = {}
max_env_avg_interv_error_value = -np.inf
max_env_avg_interv_error_key   = None
distance_err                   = 'wass'

for lenv in A_ll:

    Dll_noise      = lenv.sample(num_llsamples)[0]
    ll_environment = mut.get_exogenous_distribution(Dll_noise)

    for henv in A_hl:
        Dhl_noise      = henv.sample(num_hlsamples)[0]
        hl_environment = mut.get_exogenous_distribution(Dhl_noise)

        total_ui_error = 0
        num_distros    = len(Ill)

        n, m  = len(LLmodels[None].endogenous_vars), len(HLmodels[None].endogenous_vars)

        T     = mut.sample_stoch_matrix(n, m)

        for iota in Ill:
            llcm   = LLmodels[iota]
            hlcm   = HLmodels[omega[iota]]
            llmech = llcm.compute_mechanism()
            hlmech = hlcm.compute_mechanism()
            error  = mut.ui_error_dist(distance_err, lenv, henv, llmech, hlmech, T)

            total_ui_error += error

        avg_interv_error = total_ui_error/num_distros

        if avg_interv_error > max_env_avg_interv_error_value:
            max_env_avg_interv_error_value = avg_interv_error
            max_env_avg_interv_error_key   = (lenv, henv)

        abstraction_errors[str(T)] = avg_interv_error
        abstraction_env_errors['ll: '+str(ll_environment.means_)+' hl: '+str(hl_environment.means_)] = avg_interv_error

/Users/giofelekis/opt/anaconda3/envs/erica/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [11]:
max_tau   = max(abstraction_errors, key=abstraction_errors.get)
max_error = abstraction_errors[max_tau]

print(f"Abstraction: {max_tau}, Error: {max_error}")
print('==============================================================================' )
max_lenv = max_env_avg_interv_error_key[0]
max_henv = max_env_avg_interv_error_key[1]

print(f"max LL mean vector = {max_lenv.means_}")
print(f"max LL covariance = {max_lenv.covariances_}")
print( )

print(f"max HL mean vector = {max_henv.means_}")
print(f"max HL covariance = {max_henv.covariances_}")
print('==============================================================================' )
print(f"max environment, average interventional abstraction error = {max_env_avg_interv_error_value}")

Abstraction: [[0.21761458 0.78238542]
 [0.75095088 0.24904912]
 [0.86664525 0.13335475]], Error: 1.080267499893765
max LL mean vector = [[0.03909886 0.02270429 0.149256  ]]
max LL covariance = [[[0.82603265 0.         0.        ]
  [0.         2.03002425 0.        ]
  [0.         0.         0.84745966]]]

max HL mean vector = [[ 0.07752375 -0.03284999]]
max HL covariance = [[[1.03175598 0.        ]
  [0.         0.77764308]]]
max environment, average interventional abstraction error = 1.080267499893765


In [41]:
experiments     = ['synth1_gnd', 'little_lucas']

for experiment in experiments:

    # Define the radius of the Wasserstein balls (epsilon, delta) and the size for both models.
    epsilon         = params.radius[experiment][0]
    ll_num_envs     = params.n_envs[experiment][0]

    delta           = params.radius[experiment][1]
    hl_num_envs     = params.n_envs[experiment][1]

    # Define the number of samples per environment. Currently every environment has the same number of samples
    num_llsamples   = params.n_samples[experiment][0]
    num_hlsamples   = params.n_samples[experiment][1]

    Dll = mut.load_samples(experiment)[None][0] 
    Gll = mut.load_ll_model(experiment)[0]
    Ill = mut.load_ll_model(experiment)[1]


    Dhl = mut.load_samples(experiment)[None][1] 
    Ghl = mut.load_hl_model(experiment)[0]
    Ihl = mut.load_hl_model(experiment)[1]

    omega = mut.load_omega_map(experiment)

    ll_coeffs = mut.get_coefficients(Dll, Gll)
    hl_coeffs = mut.get_coefficients(Dhl, Ghl) 
    num_experiments = 100


In [42]:
# Define initial values (replace with actual values from your environment)
mu_L    = torch.from_numpy(mu_U_ll_hat)
Sigma_L = torch.from_numpy(Sigma_U_ll_hat)

mu_H    = torch.from_numpy(mu_U_hl_hat)
Sigma_H = torch.from_numpy(Sigma_U_hl_hat)

l = mu_L.shape[0]
h = mu_H.shape[0]

num_intervs = len(Ill)

# Define T as a tensor with requires_grad=True for automatic differentiation
T = torch.randn(h, l, requires_grad=True)

# Define the optimizer
optimizer = torch.optim.Adam([T], lr=0.01)
#optimizer = torch.optim.SGD([T], lr=0.01)

# Number of iterations (epochs)
num_epochs = 100  

# List to store the objective values at each epoch
objective_values = []

for epoch in range(num_epochs):
    # Initialize the objective for this iteration
    objective = 0
    
    # Compute the objective function as the expectation over samples
    for i, iota in enumerate(Ill):
        L_i = torch.from_numpy(LLmodels[iota].compute_mechanism())
        H_i = torch.from_numpy(HLmodels[omega[iota]].compute_mechanism())
        
        # 1st term: || T (L_i * mu_L) - (H_i * mu_H) ||_2^2
        L_i_mu_L = L_i @ mu_L  # Result: (m,)
        H_i_mu_H = H_i @ mu_H  # Result: (n,)
        
        term1 = torch.norm(T.float() @ L_i_mu_L.float() - H_i_mu_H.float())**2  # Now this is dimensionally consistent
        
        # 2nd term: Tr(T L_i Sigma_L L_i^T T^T)
        term2 = torch.trace(T.float() @ L_i.float() @ Sigma_L.float() @ L_i.T.float() @ T.T.float())
        
        # 3rd term: Tr(H_i Sigma_H H_i^T)
        term3 = torch.trace(H_i.float() @ Sigma_H.float() @ H_i.T.float())
        
        # Ensure positive-definiteness for Cholesky decomposition
        L_i_Sigma_L = T.float() @ L_i.float() @ Sigma_L.float() @ L_i.T.float() @ T.T.float()
        H_i_Sigma_H = H_i.float() @ Sigma_H.float() @ H_i.T.float()
        
        # 4th term: -2 * || (T L_i Sigma_L L_i^T T^T)^(1/2) * (H_i Sigma_H H_i^T)^(1/2) ||_*
        term4 = -2 * torch.norm(torch.linalg.cholesky(L_i_Sigma_L) @ torch.linalg.cholesky(H_i_Sigma_H), 'nuc')

        # Sum up terms
        objective += term1 + term2 + term3 + term4

    # Average the objective over all interventions
    objective /= num_intervs

    # Append the objective value at this epoch
    objective_values.append(objective.item())

    # Compute gradients (subgradients for nuclear norm)
    optimizer.zero_grad()  # Zero out previous gradients
    objective.backward()  # Backpropagate to compute gradients

    print(f"Epoch {epoch+1}/{num_epochs}, Objective: {objective.item()}")

    # Perform one step of gradient descent
    optimizer.step()  # Update T using the optimizer

# After the loop, plot the objective values per epoch
plt.plot(range(num_epochs), objective_values, label='Objective per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Objective Value')
plt.title('Optimization Progress')
plt.grid(True)
plt.legend()
plt.show()

print("Final T:", T)

KeyError: <operations.Intervention object at 0x19850ce90>

In [43]:
LLmodels = {}
for iota in Ill:
    LLmodels[iota] = lanm.LinearAddSCM(Gll, ll_coeffs, iota)
    
HLmodels, Dhl_samples = {}, {}
for eta in Ihl:
    HLmodels[eta] = lanm.LinearAddSCM(Ghl, hl_coeffs, eta)

In [112]:
# # Define the update functions
# def update_mu_L(LLmodels, HLmodels, mu_H, lambda_eps, mu_U_ll_hat):
#     N = len(LLmodels)
#     E_LL = sum(LL_i.T @ LL_i for LL_i in LLmodels) / N
#     E_LH_mu_H = sum(LL_i.T @ HL_i @ mu_H for LL_i, HL_i in zip(LLmodels, HLmodels)) / N
#     reg_term = (lambda_eps / 2) * np.eye(mu_U_ll_hat.shape[0])
#     mu_L_expr = np.linalg.inv(E_LL + reg_term) @ (E_LH_mu_H + (lambda_eps / 2) * mu_U_ll_hat)
#     return mu_L_expr

# def update_mu_H(HLmodels, LLmodels, mu_L, lambda_delta, mu_U_hl_hat):
#     N = len(HLmodels)
#     E_HH = sum(HL_iota.T @ HL_iota for HL_iota in HLmodels) / N
#     E_HL_mu_L = sum(HL_iota.T @ LL_iota @ mu_L for LL_iota, HL_iota in zip(LLmodels, HLmodels)) / N
#     reg_term = (lambda_delta / 2) * np.eye(mu_U_hl_hat.shape[0])
#     mu_H_expr = np.linalg.inv(E_HH + reg_term) @ (E_HL_mu_L + (lambda_delta / 2) * mu_U_hl_hat)
#     return mu_H_expr

# def update_Sigma_L(LLmodels, lambda_eps, Sigma_U_ll_hat, lambda_Q, Sigma_L_k):
#     # Ensure that lambda_eps is a scalar or a single value
#     if np.isscalar(lambda_eps) or lambda_eps.size == 1:
#         lambda_eps_value = lambda_eps if np.isscalar(lambda_eps) else lambda_eps.item()
#         if lambda_eps_value > 0:
#             # Check shapes of LLmodels and lambda_Q
#             LL_sum = np.sum([LL_i @ LL_i.T for LL_i in LLmodels], axis=0)  # Should result in (4, 4) if each LL_i is (4, 3)
#             lambda_Q_sum = np.sum(lambda_Q, axis=0)  # Ensure this matches the expected shape
            
#             # Make sure lambda_Q_sum is shaped correctly for the operation
#             if lambda_Q_sum.shape != LL_sum.shape:
#                 raise ValueError(f"Shape mismatch: LL_sum shape {LL_sum.shape} and lambda_Q_sum shape {lambda_Q_sum.shape}")

#             Sigma_L_updated = np.linalg.inv(LL_sum + lambda_eps_value * np.eye(Sigma_L_k.shape[0])) @ (
#                 lambda_eps_value * Sigma_U_ll_hat + lambda_Q_sum
#             )
#         else:
#             Sigma_L_updated = np.zeros_like(Sigma_L_k)  # Handle the case where the condition is not met
#     else:
#         raise ValueError("lambda_eps should be a scalar or a single value")

#     return Sigma_L_updated

# def update_Sigma_H(HLmodels, lambda_delta, Sigma_U_hl_hat, lambda_W, Sigma_H_k):
#     if np.isscalar(lambda_delta) or lambda_delta.size == 1:
#         lambda_delta_value = lambda_delta if np.isscalar(lambda_delta) else lambda_delta.item()
#         if lambda_delta_value > 0:
#             Sigma_H_updated = np.linalg.inv(np.sum(HLmodels, axis=0) + lambda_delta_value * np.eye(Sigma_H_k.shape[0])) @ (lambda_delta_value * Sigma_U_hl_hat + np.sum(lambda_W, axis=0))
#         else:
#             Sigma_H_updated = np.zeros_like(Sigma_H_k)
#     else:
#         raise ValueError("lambda_delta should be a scalar or a single value")
#     return Sigma_H_updated

# def update_Q_i(Q_i_k, lambda_Q_i, rho_Q, alpha):
#     norm_Q = np.linalg.norm(Q_i_k, 'fro')
#     if norm_Q > alpha:
#         Q_i_updated = (1 - (alpha / norm_Q)) * (Q_i_k - (1 / rho_Q) * lambda_Q_i)
#     else:
#         Q_i_updated = np.zeros_like(Q_i_k)
#     return Q_i_updated

# def update_W_i(W_i_k, lambda_W_i, rho_W, alpha):
#     norm_W = np.linalg.norm(W_i_k, 'fro')
#     if norm_W > alpha:
#         W_i_updated = (1 - (alpha / norm_W)) * (W_i_k - (1 / rho_W) * lambda_W_i)
#     else:
#         W_i_updated = np.zeros_like(W_i_k)
#     return W_i_updated

# def update_lambda_Q(lambda_Q_i_k, rho_Q, Q_i_k_plus_1, LL_i, Sigma_L_k_plus_1):
#     term = np.sqrt(LL_i @ Sigma_L_k_plus_1 @ LL_i.T)
#     lambda_Q_i_updated = lambda_Q_i_k + rho_Q * (Q_i_k_plus_1 - term)
#     return lambda_Q_i_updated

# def update_lambda_W(lambda_W_i_k, rho_W, W_i_k_plus_1, HL_i, Sigma_H_k_plus_1):
#     term = np.sqrt(HL_i @ Sigma_H_k_plus_1 @ HL_i.T)
#     lambda_W_i_updated = lambda_W_i_k + rho_W * (W_i_k_plus_1 - term)
#     return lambda_W_i_updated

# def update_lambda_epsilon(lambda_eps_k, mu_L_k_plus_1, mu_U_ll_hat, Sigma_L_k_plus_1, Sigma_U_ll_hat, epsilon):
#     lambda_eps_updated = lambda_eps_k + (epsilon**2 - np.linalg.norm(mu_L_k_plus_1 - mu_U_ll_hat)**2 - np.linalg.norm(np.sqrt(Sigma_L_k_plus_1) - np.sqrt(Sigma_U_ll_hat))**2)
#     return lambda_eps_updated

# def update_lambda_delta(lambda_del_k, mu_H_k_plus_1, mu_U_hl_hat, Sigma_H_k_plus_1, Sigma_U_hl_hat, delta):
#     lambda_del_updated = lambda_del_k + (delta**2 - np.linalg.norm(mu_H_k_plus_1 - mu_U_hl_hat)**2 - np.linalg.norm(np.sqrt(Sigma_H_k_plus_1) - np.sqrt(Sigma_U_hl_hat))**2)
#     return lambda_del_updated

# def compute_monge_map(mu_a, Sigma_a, mu_b, Sigma_b):
#     # Check dimensions
#     if mu_a.shape[0] != 3 or mu_b.shape[0] != 2:
#         raise ValueError("Mean vectors must be of shape (3,) for Gaussian A and (2,) for Gaussian B.")
    
#     if Sigma_a.shape != (3, 3) or Sigma_b.shape != (2, 2):
#         raise ValueError("Covariance matrices must be of shape (3, 3) for Gaussian A and (2, 2) for Gaussian B.")

#     # Compute the square root of the covariance matrices
#     Sigma_a_sqrt = np.linalg.cholesky(Sigma_a)

#     # Use a 2D projection from 3D to 2D, we need a method to match the dimensions
#     # Here we assume a simple linear map for projection
#     A = Sigma_a_sqrt[:2, :2]  # Take the first two rows/columns for the projection

#     # Compute the transformation matrix
#     A_inv = np.linalg.inv(A)
#     B_inv = np.linalg.inv(Sigma_b)
    
#     # Calculate the optimal transformation using the covariance matrices
#     transformation_matrix = A_inv @ Sigma_b @ A_inv

#     # Compute the inverse square root of the transformation matrix
#     transformation_matrix_sqrt_inv = np.linalg.inv(np.linalg.cholesky(transformation_matrix))

#     def T(x):
#         # Ensure the input x is in the expected shape (3,)
#         if x.shape[0] != 3:
#             raise ValueError("Input x must be a 3D vector (shape: (3,)).")
#         # Map the 3D vector to 2D
#         return transformation_matrix_sqrt_inv @ (x - mu_a)[:2] + mu_b

#     return T

In [20]:
# # Optimization Parameters Initialization
# num_iterations = 100000  # Number of iterations for optimization
# rho_Q = 1.0  # Penalty parameter for Q updates
# rho_W = 1.0  # Penalty parameter for W updates
# alpha = 0.5  # Proximal parameter
# epsilon = 0.1  # Epsilon constraint value
# delta = 0.1  # Delta constraint value

# # Example initialization (replace these with actual data)
# LLmodels = [np.random.rand(4, 3) for _ in range(5)]  # Example L_i matrices
# HLmodels = [np.random.rand(4, 2) for _ in range(5)]  # Example H_i matrices
# mu_U_ll_hat = np.random.rand(3)  # Target mean for mu_L
# mu_U_hl_hat = np.random.rand(2)  # Target mean for mu_H
# Sigma_U_ll_hat = np.eye(3)  # Target covariance for Sigma_L
# Sigma_U_hl_hat = np.eye(2)  # Target covariance for Sigma_H

# # Initialize variables
# mu_L = np.random.rand(3)  # Initial estimate for mu_L
# mu_H = np.random.rand(2)  # Initial estimate for mu_H
# Sigma_L = np.eye(3)  # Initial estimate for Sigma_L
# Sigma_H = np.eye(2)  # Initial estimate for Sigma_H
# Q_vars = [np.random.rand(4, 4) for _ in range(5)]  # Initial Q matrices
# W_vars = [np.random.rand(4, 4) for _ in range(5)]  # Initial W matrices
# lambda_Q = [np.zeros((4, 4)) for _ in range(5)]  # Initial lambda_Q
# lambda_W = [np.zeros((4, 4)) for _ in range(5)]  # Initial lambda_W
# lambda_eps = 0.0  # Initial lambda_eps
# lambda_del = 0.0  # Initial lambda_delta

# # Optimization Loop
# for k in range(num_iterations):
#     # Update mu_L and mu_H
#     mu_L = update_mu_L(LLmodels, HLmodels, mu_H, lambda_eps, mu_U_ll_hat)
#     mu_H = update_mu_H(HLmodels, LLmodels, mu_L, lambda_delta, mu_U_hl_hat)

#     # Update Sigma_L and Sigma_H
#     Sigma_L = Sigma_U_ll_hat #update_Sigma_L(LLmodels, rho_Q, Sigma_U_ll_hat, lambda_Q, Sigma_L)
#     Sigma_H = Sigma_U_hl_hat #update_Sigma_H(HLmodels, rho_W, Sigma_U_hl_hat, lambda_W, Sigma_H)

#     # Update Q and W
#     for iota in range(len(Q_vars)):
#         Q_vars[iota] = update_Q_i(Q_vars[iota], lambda_Q[iota], rho_Q, alpha)
#         W_vars[iota] = update_W_i(W_vars[iota], lambda_W[iota], rho_W, alpha)

#     # Update lambda multipliers
#     for iota in range(len(Q_vars)):
#         lambda_Q[iota] = update_lambda_Q(lambda_Q[iota], rho_Q, Q_vars[iota], LLmodels[iota], Sigma_L)
#         lambda_W[iota] = update_lambda_W(lambda_W[iota], rho_W, W_vars[iota], HLmodels[iota], Sigma_H)

#     # Update lambda_eps and lambda_delta
#     lambda_eps = update_lambda_epsilon(lambda_eps, mu_L, mu_U_ll_hat, Sigma_L, Sigma_U_ll_hat, epsilon)
#     lambda_del = update_lambda_delta(lambda_del, mu_H, mu_U_hl_hat, Sigma_H, Sigma_U_hl_hat, delta)

# # Print the final results
# print("Final mu_L:", mu_L)
# print("Final Sigma_L:\n", Sigma_L)
# print("Final mu_H:", mu_H)
# print("Final Sigma_H:\n", Sigma_H)
# print( )
# tau = compute_monge_map(mu_L, Sigma_L, mu_H, Sigma_H)
# x_sample = np.array([1.5, 2.5, 3.5])
# mapped_point = tau(x_sample)

# print("Mapped point:", mapped_point)

In [159]:
def update_Sigma_H(Sigma_H_half, LLmodels, Sigma_L, HLmodels, lambda_param):
    Sigma_H_final = torch.zeros_like(Sigma_H_half, dtype=torch.complex64)  # Ensure dtype is complex64
    for n, iota in enumerate(Ill):
        L_i = torch.from_numpy(LLmodels[iota].compute_mechanism()).float()  # Convert to float32
        V_i = T @ L_i  # Matrix multiplication, ensure V_i is float32
        H_i = torch.from_numpy(HLmodels[omega[iota]].compute_mechanism()).float()  # Convert to float32

        # Ensure Sigma_H_half is float32 for matrix multiplication
        Sigma_H_half = Sigma_H_half.float()

        # Compute term_sqrt with cholesky
        term_sqrt = torch.cholesky(torch.matmul(H_i, torch.matmul(Sigma_H_half, H_i.T)))

        # Proximal operator
        prox_Sigma_H_half = torch.matmul(prox_operator(term_sqrt, lambda_param), prox_operator(term_sqrt, lambda_param).T)

        # Matrix multiplication and inversion
        hl_term = torch.matmul(torch.matmul(torch.inverse(H_i), prox_Sigma_H_half), torch.inverse(H_i).T)

        # Convert Sigma_L to complex64 for compatibility with complex hl_term
        Sigma_L_complex = Sigma_L.to(torch.complex64)  # Convert Sigma_L to complex64

        # Convert V_i and V_i.T to complex64 to match Sigma_L_complex
        V_i_complex = V_i.to(torch.complex64)  # Convert V_i to complex64
        V_i_T_complex = V_i.T.to(torch.complex64)  # Convert V_i.T to complex64

        # Compute ll_term with the complex tensors
        ll_term = torch.norm(torch.cholesky(torch.matmul(V_i_complex, torch.matmul(Sigma_L_complex, V_i_T_complex))), p='fro')

        # Ensure hl_term is complex to match ll_term
        hl_term = hl_term.to(torch.complex64)  # Convert hl_term to complex64

        # Update Sigma_H_final by scaling ll_term with hl_term (instead of matmul)
        Sigma_H_final += ll_term * hl_term  # Scale ll_term by the scalar hl_term

    # Final update
    Sigma_H_final *= - (2 / n)
    Sigma_H_final = diagonalize(Sigma_H_final)

    return Sigma_H_final


In [160]:
mu_L, Sigma_L, mu_H, Sigma_H = optimize(mu_L, Sigma_L, mu_H, Sigma_H, LLmodels,
                                        HLmodels, hat_mu_L, hat_Sigma_L, hat_mu_H, hat_Sigma_H,
                                        lambda_L=0.1, lambda_H=0.1, lambda_param=0.1, eta=0.01, max_iter=100)

_LinAlgError: cholesky: The factorization could not be completed because the input is not positive-definite (the leading minor of order 1 is not positive-definite).

In [152]:
def update_mu_L(mu_L, mu_H, LLmodels, HLmodels, lambda_L, hat_mu_L, eta):
    grad_mu_L = torch.zeros_like(mu_L, dtype=torch.float32) 
    for n, iota in enumerate(Ill):
        L_i = torch.from_numpy(LLmodels[iota].compute_mechanism()).float() 
        V_i = T @ L_i  
        H_i = torch.from_numpy(HLmodels[omega[iota]].compute_mechanism()).float() 

        grad_mu_L += torch.matmul(V_i.T, torch.matmul(V_i, mu_L.float()) - torch.matmul(H_i, mu_H.float())) 
    
    grad_mu_L = (2 / n) * grad_mu_L - 2 * lambda_L * (mu_L - hat_mu_L)
    mu_L += eta * grad_mu_L
    return mu_L

def update_mu_H(mu_L, mu_H, LLmodels, HLmodels, lambda_H, hat_mu_H, eta):
    grad_mu_H = torch.zeros_like(mu_H, dtype=torch.float32)  
    for n, iota in enumerate(Ill):
        L_i = torch.from_numpy(LLmodels[iota].compute_mechanism()).float()  
        V_i = T @ L_i  
        H_i = torch.from_numpy(HLmodels[omega[iota]].compute_mechanism()).float()  

        grad_mu_H -= torch.matmul(H_i.T, torch.matmul(V_i, mu_L.float()) - torch.matmul(H_i, mu_H.float()))
    
    grad_mu_H = (2 / n) * grad_mu_H - 2 * lambda_H * (mu_H - hat_mu_H)
    
    mu_H += eta * grad_mu_H
    return mu_H

def update_Sigma_L_half(Sigma_L, LLmodels, lambda_L, hat_Sigma_L, eta):
    grad_Sigma_L = torch.zeros_like(Sigma_L)
    
    # Term 1: (2/n) * sum_i(V_i^T * V_i)
    term1 = torch.zeros_like(Sigma_L)
    for n, iota in enumerate(Ill):
        L_i = torch.from_numpy(LLmodels[iota].compute_mechanism())
        V_i = T @ L_i.float()
        term1 += torch.matmul(V_i.T, V_i)

    # Term 2: -2 * lambda_L * (Sigma_L^(1/2) - hat_Sigma_L^(1/2)) * Sigma_L^(-1/2)
    Sigma_L_sqrt = torch.cholesky(Sigma_L)  # Compute the square root of Sigma_L
    #Sigma_L_sqrt = torch.linalg.matrix_power(Sigma_L, 0.5)

    hat_Sigma_L_sqrt = torch.cholesky(hat_Sigma_L)  # Compute the square root of hat_Sigma_L

    term2 = -2 * lambda_L * (Sigma_L_sqrt - hat_Sigma_L_sqrt) @ torch.inverse(Sigma_L_sqrt)

    # Combine terms
    grad_Sigma_L = (2 / n) * term1 + term2

    # Update Sigma_L
    Sigma_L_half = Sigma_L + eta * grad_Sigma_L
    return Sigma_L_half

# Proximal operator of a matrix frobenious norm
def prox_operator(A, lambda_param):
    frobenius_norm = torch.norm(A, p='fro')
    scaling_factor = torch.max(1 - lambda_param / frobenius_norm, torch.zeros_like(frobenius_norm))
    return scaling_factor * A

def diagonalize(A):
    # Get eigenvalues and eigenvectors using the new method
    eigvals, eigvecs = torch.linalg.eig(A)  # This will return complex values
    
    # If you need only real eigenvalues, we take the real part
    eigvals_real = eigvals.real  # We only need the real part if A is symmetric
    
    # Convert the diagonal matrix to complex to match the dtype of eigvecs
    D = torch.diag(eigvals_real).to(torch.complex64)  # Ensure D is of complex type
    
    # Return the diagonalized matrix
    return torch.matmul(torch.matmul(eigvecs, D), eigvecs.T.conj())


def update_Sigma_L(Sigma_L_half, LLmodels, Sigma_H, HLmodels, lambda_param):
    Sigma_L_final = torch.zeros_like(Sigma_L_half, dtype=torch.float32)  # Ensure dtype is float32
    for n, iota in enumerate(Ill):
        # Convert L_i and H_i to float32
        L_i = torch.from_numpy(LLmodels[iota].compute_mechanism()).float()  # Convert to float32
        V_i = T @ L_i  # Matrix multiplication, ensure V_i is float32
        H_i = torch.from_numpy(HLmodels[omega[iota]].compute_mechanism()).float()  # Convert to float32

        # Make sure Sigma_L_half is float32 for matrix multiplication
        Sigma_L_half = Sigma_L_half.float()

        # Perform matrix multiplication with consistent dtype (float32)
        term_sqrt = torch.cholesky(torch.matmul(V_i, torch.matmul(Sigma_L_half, V_i.T)))  # Ensure all are float32

        # Proximal operator
        prox_Sigma_L_half = torch.matmul(prox_operator(term_sqrt, lambda_param), prox_operator(term_sqrt, lambda_param).T)

        # Pseudo-inverse of V_i and other matrix multiplications
        ll_term = torch.matmul(torch.matmul(torch.linalg.pinv(V_i), prox_Sigma_L_half), torch.linalg.pinv(V_i).T)

        # Ensure Sigma_H is float32 before matrix multiplication and Cholesky
        Sigma_H = Sigma_H.float()  # Explicitly cast Sigma_H to float32

        # Compute H_i * Sigma_H * H_i^T and take its Cholesky
        hl_term = torch.norm(torch.cholesky(torch.matmul(H_i, torch.matmul(Sigma_H, H_i.T).float())), p='fro')

        # Update Sigma_L_final
        Sigma_L_final += ll_term * hl_term

    # Final update
    Sigma_L_final *= - (2 / n)
    Sigma_L_final = diagonalize(Sigma_L_final)

    return Sigma_L_final


def update_Sigma_H_half(Sigma_H, HLmodels, lambda_H, hat_Sigma_H, eta):
    grad_Sigma_H = torch.zeros_like(Sigma_H)
    
    # Term 1: (2/n) * sum_i(H_i^T * H_i)
    term1 = torch.zeros_like(Sigma_H)
    for n, iota in enumerate(Ill):
        H_i = torch.from_numpy(HLmodels[omega[iota]].compute_mechanism())

        term1 += torch.matmul(H_i.T, H_i)

    # Term 2: -2 * lambda_H * (Sigma_H^(1/2) - hat_Sigma_H^(1/2)) * Sigma_H^(-1/2)
    Sigma_H_sqrt     = torch.cholesky(Sigma_H)  # Compute the square root of Sigma_H
    hat_Sigma_H_sqrt = torch.cholesky(hat_Sigma_H)  # Compute the square root of hat_Sigma_H

    term2 = -2 * lambda_H * (Sigma_H_sqrt - hat_Sigma_H_sqrt) @ torch.inverse(Sigma_H_sqrt)

    # Combine terms
    grad_Sigma_H = (2 / n) * term1 + term2

    # Update Sigma_H
    Sigma_H_half = Sigma_H + eta * grad_Sigma_H
    return Sigma_H_half


def update_Sigma_H(Sigma_H_half, LLmodels, Sigma_L, HLmodels, lambda_param):
    Sigma_H_final = torch.zeros_like(Sigma_H_half)
    for n, iota in enumerate(Ill):
        L_i = torch.from_numpy(LLmodels[iota].compute_mechanism())
        V_i = T @ L_i.float()
        H_i = torch.from_numpy(HLmodels[omega[iota]].compute_mechanism())

        term_sqrt         = torch.cholesky(torch.matmul(H_i, torch.matmul(Sigma_H_half, H_i.T))) 
        prox_Sigma_H_half = torch.matmul(prox_operator(term_sqrt, lambda_param), prox_operator(term_sqrt, lambda_param).T) 
        hl_term           = torch.matmul(torch.matmul(torch.inverse(H_i), prox_Sigma_H_half), torch.inverse(H_i).T)  

        #Compute H_i * Sigma_H * H_i^T and take its Cholesky
        ll_term = torch.norm(torch.cholesky(torch.matmul(V_i, torch.matmul(Sigma_L, V_i.T))), p='fro')

        #Update Sigma_L_final
        Sigma_H_final += torch.matmul(ll_term, hl_term)
    
    Sigma_H_final *= - (2 / n)
    Sigma_H_final  = diagonalize(Sigma_H_final)
    
    return Sigma_H_final


In [150]:
def optimize(mu_L, Sigma_L, mu_H, Sigma_H, LLmodels, HLmodels, hat_mu_L, hat_Sigma_L, hat_mu_H, hat_Sigma_H, lambda_L, lambda_H, lambda_param, eta, max_iter):
    for t in range(max_iter):
        # Update mu_L and mu_H
        mu_L = update_mu_L(mu_L, mu_H, LLmodels, HLmodels, lambda_L, hat_mu_L, eta)
        mu_H = update_mu_H(mu_L, mu_H, LLmodels, HLmodels, lambda_H, hat_mu_H, eta)
        
        # Update Sigma_L (t+1/2, t)
        Sigma_L_half = update_Sigma_L_half(Sigma_L, LLmodels, lambda_L, hat_Sigma_L, eta)
        Sigma_L      = update_Sigma_L(Sigma_L_half, LLmodels, Sigma_H, HLmodels, lambda_param)

                
        # Update Sigma_H (t+1/2, t)
        Sigma_H_half = update_Sigma_H_half(Sigma_H, HLmodels, lambda_H, hat_Sigma_H, eta)
        Sigma_H      = update_Sigma_H(Sigma_H_half, LLmodels, Sigma_L, HLmodels, lambda_param)
        
        if t % 10 == 0:
            print(f"Iteration {t} - mu_L: {mu_L}, mu_H: {mu_H}, Sigma_L: {Sigma_L}, Sigma_H: {Sigma_H}")
    
    return mu_L, Sigma_L, mu_H, Sigma_H

In [120]:
# Example usage
n = 10  # Number of data points
l = 5  # Dimension of mu_L and Sigma_L
h = 5  # Dimension of mu_H and Sigma_H

mu_L    = torch.from_numpy(mu_U_ll_hat)
Sigma_L = torch.from_numpy(Sigma_U_ll_hat)

mu_H    = torch.from_numpy(mu_U_hl_hat)
Sigma_H = torch.from_numpy(Sigma_U_hl_hat)

l = mu_L.shape[0]
h = mu_H.shape[0]

# Given estimates (mu_L, Sigma_L, mu_H, Sigma_H)
hat_mu_L = torch.randn(l)  # Estimate of mu_L
hat_Sigma_L = torch.eye(l)  # Estimate of Sigma_L (identity for simplicity)

hat_mu_H = torch.randn(h)  # Estimate of mu_H
hat_Sigma_H = torch.eye(h)  # Estimate of Sigma_H (identity for simplicity)

In [121]:
T = torch.from_numpy(np.random.randn(2, 3)).float()

In [122]:
mu_L, Sigma_L, mu_H, Sigma_H = optimize(mu_L.float(), Sigma_L.float(), mu_H.float(), Sigma_H.float(), LLmodels,
                                        HLmodels, hat_mu_L.float(), hat_Sigma_L.float(), hat_mu_H.float(), hat_Sigma_H.float(),
                                        lambda_L=0.1, lambda_H=0.1, lambda_param=0.1, eta=0.01, max_iter=100)


RuntimeError: expected scalar type Double but found Float

In [22]:
L_i.float()

tensor([[1.0000, 0.2802, 0.0607],
        [0.0000, 1.0000, 0.2167],
        [0.0000, 0.0000, 1.0000]])

In [34]:
#with grad

In [35]:
import torch

# Define the optimize function with autograd for mu_L, mu_H, Sigma_L_half, Sigma_H_half
def optimize(LLmodels, HLmodels, n, mu_L, Sigma_L, mu_H, Sigma_H, hat_mu_L, hat_Sigma_L, hat_mu_H, hat_Sigma_H, epsilon, delta, lambda_L, lambda_H, eta, max_iter):
    # Enable autograd for mu_L, mu_H, Sigma_L_half, Sigma_H_half
    mu_L.requires_grad_(True)
    mu_H.requires_grad_(True)

    Sigma_L_half = Sigma_L.clone().requires_grad_(True)  # Enable autograd for Sigma_L_half
    Sigma_H_half = Sigma_H.clone().requires_grad_(True)  # Enable autograd for Sigma_H_half

    # Sigma_L and Sigma_H do not need autograd for the final update
    Sigma_L.requires_grad_(False)
    Sigma_H.requires_grad_(False)

    for t in range(max_iter):
        # Forward pass: compute the objective function
        F_value = F_func(mu_L, Sigma_L, mu_H, Sigma_H, LLmodels, HLmodels, lambda_L, lambda_H, hat_mu_L, hat_Sigma_L, hat_mu_H, hat_Sigma_H, n, epsilon, delta)
        objective = F_value   # Total objective function

        # Backward pass: compute gradients for mu_L, mu_H, Sigma_L_half, Sigma_H_half
        objective.backward()

        # Update mu_L and mu_H using gradient ascent with autograd
        with torch.no_grad():
            mu_L += eta * mu_L.grad  # Gradient ascent for mu_L
            mu_H += eta * mu_H.grad  # Gradient ascent for mu_H

            # Zero the gradients after the update for mu_L and mu_H
            mu_L.grad.zero_()
            mu_H.grad.zero_()

        # Update Sigma_L_half and Sigma_H_half using autograd
        with torch.no_grad():
            # Update Sigma_L_half and Sigma_H_half using the gradient of F
            Sigma_L_half += eta * Sigma_L_half.grad  # Update Sigma_L_half using gradients of F
            Sigma_H_half += eta * Sigma_H_half.grad  # Update Sigma_H_half using gradients of F

            # Zero the gradients after the update for Sigma_L_half and Sigma_H_half
            Sigma_L_half.grad.zero_()
            Sigma_H_half.grad.zero_()

        # Sigma_L and Sigma_H Updates (no autograd for final update, using proximal operator)
        with torch.no_grad():
            # Final update for Sigma_L and Sigma_H
            Sigma_L_final = update_Sigma_L_final(Sigma_L_half, V, H, Sigma_H, lambda_L, lambda_param, n)
            Sigma_H_final = update_Sigma_H_final(Sigma_H_half, V, Sigma_L, H, lambda_H, lambda_param, n)
            
            # Apply the proximal operator and projections
            Sigma_L = update_Sigma_L_with_projection(Sigma_L_final)
            Sigma_H = update_Sigma_H_with_projection(Sigma_H_final)

        # Print progress
        if t % 10 == 0:
            print(f"Iteration {t}, Objective Value: {objective.item()}")

    return mu_L, Sigma_L, mu_H, Sigma_H


In [36]:
import torch

# Objective function F(mu_L, Sigma_L, mu_H, Sigma_H) calculation
def F_func(mu_L, Sigma_L, mu_H, Sigma_H, LLmodels, HLmodels, lambda_L, lambda_H, hat_mu_L, hat_Sigma_L, hat_mu_H, hat_Sigma_H, n, epsilon, delta):
    F_value = 0.0
    for i in range(n):
        # Compute terms inside the summation
        term1 = torch.norm(torch.matmul(LLmodels[i], mu_L) - torch.matmul(HLmodels[i], mu_H)) ** 2
        term2 = torch.trace(torch.matmul(LLmodels[i], torch.matmul(Sigma_L, LLmodels[i].T)))
        term3 = torch.trace(torch.matmul(HLmodels[i], torch.matmul(Sigma_H, HLmodels[i].T)))
        
        # Add terms to the objective
        F_value += term1 + term2 + term3

    # Add the regularization terms
    reg_L = lambda_L * (epsilon**2 - torch.norm(mu_L - hat_mu_L)**2 - torch.norm(torch.cholesky(Sigma_L) - torch.cholesky(hat_Sigma_L))**2)
    reg_H = lambda_H * (delta**2 - torch.norm(mu_H - hat_mu_H)**2 - torch.norm(torch.cholesky(Sigma_H) - torch.cholesky(hat_Sigma_H))**2)
    
    F_value += reg_L + reg_H
    return F_value / n

# Objective function G(Sigma_L, Sigma_H) calculation
def G_func(Sigma_L, Sigma_H, LLmodels, HLmodels, n):
    G_value = 0.0
    for i in range(n):
        # Compute the Frobenius norm of the square root of the matrices
        term1 = torch.norm(torch.cholesky(torch.matmul(LLmodels[i], torch.matmul(Sigma_L, LLmodels[i].T))), p='fro')
        term2 = torch.norm(torch.cholesky(torch.matmul(HLmodels[i], torch.matmul(Sigma_H, HLmodels[i].T))), p='fro')
        
        # Add terms to the objective
        G_value += term1 * term2

    return -2 * G_value / n

# Define the optimize function with autograd for mu_L, mu_H, Sigma_L_half, Sigma_H_half
def optimize(LLmodels, HLmodels, n, mu_L, Sigma_L, mu_H, Sigma_H, hat_mu_L, hat_Sigma_L, hat_mu_H, hat_Sigma_H, epsilon, delta, lambda_L, lambda_H, eta, max_iter):
    # Enable autograd for mu_L, mu_H, Sigma_L_half, Sigma_H_half
    mu_L.requires_grad_(True)
    mu_H.requires_grad_(True)

    Sigma_L_half = Sigma_L.clone().requires_grad_(True)  # Enable autograd for Sigma_L_half
    Sigma_H_half = Sigma_H.clone().requires_grad_(True)  # Enable autograd for Sigma_H_half

    # Sigma_L and Sigma_H do not need autograd for the final update
    Sigma_L.requires_grad_(False)
    Sigma_H.requires_grad_(False)

    for t in range(max_iter):
        # Compute the total objective (F + G)
        F_value = F_func(mu_L, Sigma_L, mu_H, Sigma_H, LLmodels, HLmodels, lambda_L, lambda_H, hat_mu_L, hat_Sigma_L, hat_mu_H, hat_Sigma_H, n, epsilon, delta)
        G_value = G_func(Sigma_L, Sigma_H, LLmodels, HLmodels, n)
        objective = F_value + G_value  # Total objective function

        # Backward pass: compute gradients for mu_L, mu_H, Sigma_L_half, Sigma_H_half
        objective.backward()

        # Update mu_L and mu_H using gradient ascent with autograd
        with torch.no_grad():
            mu_L += eta * mu_L.grad  # Gradient ascent for mu_L
            mu_H += eta * mu_H.grad  # Gradient ascent for mu_H

            # Zero the gradients after the update for mu_L and mu_H
            mu_L.grad.zero_()
            mu_H.grad.zero_()

        # Update Sigma_L_half and Sigma_H_half using autograd
        with torch.no_grad():
            # Update Sigma_L_half and Sigma_H_half using the gradient of F
            Sigma_L_half += eta * Sigma_L_half.grad  # Update Sigma_L_half using gradients of F
            Sigma_H_half += eta * Sigma_H_half.grad  # Update Sigma_H_half using gradients of F

            # Zero the gradients after the update for Sigma_L_half and Sigma_H_half
            Sigma_L_half.grad.zero_()
            Sigma_H_half.grad.zero_()

        # Sigma_L and Sigma_H Updates (no autograd for final update, using proximal operator)
        with torch.no_grad():
            # Final update for Sigma_L and Sigma_H
            Sigma_L_final = update_Sigma_L_final(Sigma_L_half, V, H, Sigma_H, lambda_L, lambda_param, n)
            Sigma_H_final = update_Sigma_H_final(Sigma_H_half, V, Sigma_L, H, lambda_H, lambda_param, n)
            
            # Apply the proximal operator and projections
            Sigma_L = update_Sigma_L_with_projection(Sigma_L_final)
            Sigma_H = update_Sigma_H_with_projection(Sigma_H_final)

        # Print progress
        if t % 10 == 0:
            print(f"Iteration {t}, Objective Value: {objective.item()}")

    return mu_L, Sigma_L, mu_H, Sigma_H


In [ ]:
# Define the optimize function with autograd for mu_L, mu_H, Sigma_L_half, Sigma_H_half
def optimize(LLmodels, HLmodels, n, mu_L, Sigma_L, mu_H, Sigma_H, hat_mu_L, hat_Sigma_L, hat_mu_H, hat_Sigma_H, epsilon, delta, lambda_L, lambda_H, eta, max_iter):
    # Enable autograd for mu_L, mu_H, Sigma_L_half, Sigma_H_half
    mu_L.requires_grad_(True)
    mu_H.requires_grad_(True)

    Sigma_L_half = Sigma_L.clone().requires_grad_(True)  # Enable autograd for Sigma_L_half
    Sigma_H_half = Sigma_H.clone().requires_grad_(True)  # Enable autograd for Sigma_H_half

    # Sigma_L and Sigma_H do not need autograd for the final update
    Sigma_L.requires_grad_(False)
    Sigma_H.requires_grad_(False)

    for t in range(max_iter):
        # Compute the total objective (F + G)
        F_value = F_func(mu_L, Sigma_L, mu_H, Sigma_H, LLmodels, HLmodels, lambda_L, lambda_H, hat_mu_L, hat_Sigma_L, hat_mu_H, hat_Sigma_H, n, epsilon, delta)
        G_value = G_func(Sigma_L, Sigma_H, LLmodels, HLmodels, n)
        objective = F_value + G_value  # Total objective function

        # Backward pass: compute gradients for mu_L, mu_H, Sigma_L_half, Sigma_H_half
        objective.backward()

        # Update mu_L and mu_H using gradient ascent with autograd
        with torch.no_grad():
            mu_L += eta * mu_L.grad  # Gradient ascent for mu_L
            mu_H += eta * mu_H.grad  # Gradient ascent for mu_H

            # Zero the gradients after the update for mu_L and mu_H
            mu_L.grad.zero_()
            mu_H.grad.zero_()

        # Update Sigma_L_half and Sigma_H_half using autograd
        with torch.no_grad():
            # Update Sigma_L_half and Sigma_H_half using the gradient of F
            Sigma_L_half += eta * Sigma_L_half.grad  # Update Sigma_L_half using gradients of F
            Sigma_H_half += eta * Sigma_H_half.grad  # Update Sigma_H_half using gradients of F

            # Zero the gradients after the update for Sigma_L_half and Sigma_H_half
            Sigma_L_half.grad.zero_()
            Sigma_H_half.grad.zero_()

        # Sigma_L and Sigma_H Updates (no autograd for final update, using proximal operator)
        with torch.no_grad():
            # Final update for Sigma_L and Sigma_H
            Sigma_L_final = update_Sigma_L_final(Sigma_L_half, V, H, Sigma_H, lambda_L, lambda_param, n)
            Sigma_H_final = update_Sigma_H_final(Sigma_H_half, V, Sigma_L, H, lambda_H, lambda_param, n)
            
            # Apply the proximal operator and projections
            Sigma_L = update_Sigma_L_with_projection(Sigma_L_final)
            Sigma_H = update_Sigma_H_with_projection(Sigma_H_final)

        # Print progress
        if t % 10 == 0:
            print(f"Iteration {t}, Objective Value: {objective.item()}")

    return mu_L, Sigma_L, mu_H, Sigma_H


In [ ]:
# Define the necessary functions using PyTorch for automatic differentiation
def F_func(mu_L, Sigma_L, mu_H, Sigma_H, LLmodels, HLmodels, lambda_L, lambda_H, hat_mu_L, hat_Sigma_L, hat_mu_H, hat_Sigma_H, n, epsilon, delta):
    term1 = 0
    term2 = 0
    term3 = 0

    # Loop to compute the sum of terms
    for i, iota in enumerate(Ill):
        L_i = torch.from_numpy(LLmodels[iota].compute_mechanism())
        V_i = T @ L_i.float()
        H_i = torch.from_numpy(HLmodels[omega[iota]].compute_mechanism())

        term1 += torch.norm(torch.matmul(V_i, mu_L) - torch.matmul(H_i, mu_H))**2 + torch.trace(torch.matmul(V_i, torch.matmul(Sigma_L, V_i.T))) + torch.trace(torch.matmul(H_i, torch.matmul(Sigma_H, H_i.T)))

    term2 = lambda_L * (epsilon**2 - torch.norm(mu_L - hat_mu_L)**2 - torch.norm(torch.cholesky(Sigma_L) - torch.cholesky(hat_Sigma_L))**2)
    term3 = lambda_H * (delta**2 - torch.norm(mu_H - hat_mu_H)**2 - torch.norm(torch.cholesky(Sigma_H) - torch.cholesky(hat_Sigma_H))**2)

    return term1 / n + term2 + term3


def G_func(Sigma_L, Sigma_H, LLmodels, HLmodels, n):
    term = 0
    for i, iota in enumerate(Ill):
        L_i = torch.from_numpy(LLmodels[iota].compute_mechanism())
        V_i = T @ L_i.float()
        H_i = torch.from_numpy(HLmodels[omega[iota]].compute_mechanism())
        term += torch.norm(torch.cholesky(torch.matmul(V_i, torch.matmul(Sigma_L, V_i.T)))) * torch.norm(torch.cholesky(torch.matmul(H_i, torch.matmul(Sigma_H, H_i.T))))
    return -2 * term / n


# Proximal operator for Sigma_L (using soft-thresholding)
def prox_Sigma_L(Sigma_L, lambda_L, LLmodels, HLmodels, Sigma_H, n):
    # Using the Frobenius norm as a soft-thresholding operator for Sigma_L
    prox = torch.zeros_like(Sigma_L)
    for i, iota in enumerate(Ill):
        L_i = torch.from_numpy(LLmodels[iota].compute_mechanism())
        V_i = T @ L_i.float()
        H_i = torch.from_numpy(HLmodels[omega[iota]].compute_mechanism())

        V_Sigma_V = torch.matmul(V_i, torch.matmul(Sigma_L, V_i.T))
        H_Sigma_H = torch.matmul(H_i, torch.matmul(Sigma_H, H_i.T))
        term = torch.norm(torch.cholesky(V_Sigma_V)) * torch.norm(torch.cholesky(H_Sigma_H))
        prox[i] = (1 - lambda_L / term) * Sigma_L[i]
    return prox


# Proximal operator for Sigma_H (using soft-thresholding)
def prox_Sigma_H(Sigma_H, lambda_H, LLmodels, HLmodels, Sigma_L, n):
    prox = torch.zeros_like(Sigma_H)
    for i, iota in enumerate(Ill):
        L_i = torch.from_numpy(LLmodels[iota].compute_mechanism())
        V_i = T @ L_i.float()
        H_i = torch.from_numpy(HLmodels[omega[iota]].compute_mechanism())
        
        V_Sigma_V = torch.matmul(V_i, torch.matmul(Sigma_L, V_i.T))
        H_Sigma_H = torch.matmul(H_i, torch.matmul(Sigma_H, H_i.T))
        term = torch.norm(torch.cholesky(V_Sigma_V)) * torch.norm(torch.cholesky(H_Sigma_H))
        prox[i] = (1 - lambda_H / term) * Sigma_H[i]
    return prox


# Optimization loop using autograd and PyProximal (maximize using gradient ascent)
def optimize11(LLmodels, HLmodels, n, mu_L, Sigma_L, mu_H, Sigma_H, hat_mu_L, hat_Sigma_L, hat_mu_H, hat_Sigma_H, epsilon, delta, lambda_L, lambda_H, eta, max_iter):
    mu_L.requires_grad_(True)  # Enable autograd for mu_L
    Sigma_L.requires_grad_(True)  # Enable autograd for Sigma_L
    mu_H.requires_grad_(True)  # Enable autograd for mu_H
    Sigma_H.requires_grad_(True)  # Enable autograd for Sigma_H

    for t in range(max_iter):
        # Forward pass: compute the objective function
        F_value = F_func(mu_L, Sigma_L, mu_H, Sigma_H, LLmodels, HLmodels, lambda_L, lambda_H, hat_mu_L, hat_Sigma_L, hat_mu_H, hat_Sigma_H, n, epsilon, delta)
        G_value = G_func(Sigma_L, Sigma_H, LLmodels, HLmodels, n)
        objective = F_value + G_value  # Total objective function

        # Backward pass: compute gradients (automatically using PyTorch's autograd)
        objective.backward()

        # Update the parameters using gradient ascent with autograd
        with torch.no_grad():
            mu_L += eta * mu_L.grad  # Ascent for mu_L
            mu_H += eta * mu_H.grad  # Ascent for mu_H
            Sigma_L += eta * Sigma_L.grad  # Ascent for Sigma_L
            Sigma_H += eta * Sigma_H.grad  # Ascent for Sigma_H

            # Apply the proximal operator to Sigma_L and Sigma_H (e.g., for regularization)
            Sigma_L = prox_Sigma_L(Sigma_L, lambda_L, LLmodels, HLmodels, Sigma_H, n)
            Sigma_H = prox_Sigma_H(Sigma_H, lambda_H, LLmodels, HLmodels, Sigma_L, n)

            # Zero the gradients after the update
            mu_L.grad.zero_()
            mu_H.grad.zero_()
            Sigma_L.grad.zero_()
            Sigma_H.grad.zero_()

        # Print progress
        if t % 10 == 0:
            print(f"Iteration {t}, Objective Value: {objective.item()}")

    return mu_L, Sigma_L, mu_H, Sigma_H

# Run optimization
mu_L, Sigma_L, mu_H, Sigma_H = optimize(LLmodels, HLmodels, n, mu_L.float(), Sigma_L.float(), mu_H.float(), Sigma_H.float(), hat_mu_L.float(), hat_Sigma_L.float(), hat_mu_H.float(), hat_Sigma_H.float(), epsilon, delta, lambda_L, lambda_H, eta, max_iter)


### TODO:

1. Apply LLmodels and  HLmodels to the current closed-form and autograd implementations.

2. Look for libraries for proximal operators to incorporate in autograd.

3. Check whether I can run autograd direct even for the non-smooth term.

4. Merge T descent with theta_l, theta_h ascent.